# Introduction
This is singletonly's notebook for Stanford Ribonanza RNA Folding Competition.  I am taking this Kaggle competition as an opportunity to apply what I have learnt in data science, and also further my understanding of other hard sciences (biology) in this case.

My expectations for myself include:
1. Conduct EDA that uncovers pattern in the data
2. Build 3 models in increasing complexity
3. Submit my results to get a gauge of what I need to improve

In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

# view all columns in df
pd.set_option('display.max_columns', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## EDA

In [3]:
train_df = pd.read_csv('/kaggle/input/stanford-ribonanza-rna-folding/train_data.csv')

In [7]:
train_df.head()

,sequence_id,sequence,experiment_type,dataset_name,reads,signal_to_noise,SN_filter,reactivity_0001,reactivity_0002,reactivity_0003,reactivity_0004,reactivity_0005,reactivity_0006,reactivity_0007,reactivity_0008,reactivity_0009,reactivity_0010,reactivity_0011,reactivity_0012,reactivity_0013,reactivity_0014,reactivity_0015,reactivity_0016,reactivity_0017,reactivity_0018,reactivity_0019,reactivity_0020,reactivity_0021,reactivity_0022,reactivity_0023,reactivity_0024,reactivity_0025,reactivity_0026,reactivity_0027,reactivity_0028,reactivity_0029,reactivity_0030,reactivity_0031,reactivity_0032,reactivity_0033,reactivity_0034,reactivity_0035,reactivity_0036,reactivity_0037,reactivity_0038,reactivity_0039,reactivity_0040,reactivity_0041,reactivity_0042,reactivity_0043,reactivity_0044,reactivity_0045,reactivity_0046,reactivity_0047,reactivity_0048,reactivity_0049,reactivity_0050,reactivity_0051,reactivity_0052,reactivity_0053,reactivity_0054,reactivity_0055,reactivity_0056,reactivity_0057,reactivity_0058,reactivity_0059,reactivity_0060,reactivity_0061,reactivity_0062,reactivity_0063,reactivity_0064,reactivity_0065,reactivity_0066,reactivity_0067,reactivity_0068,reactivity_0069,reactivity_0070,reactivity_0071,reactivity_0072,reactivity_0073,reactivity_0074,reactivity_0075,reactivity_0076,reactivity_0077,reactivity_0078,reactivity_0079,reactivity_0080,reactivity_0081,reactivity_0082,reactivity_0083,reactivity_0084,reactivity_0085,reactivity_0086,reactivity_0087,reactivity_0088,reactivity_0089,reactivity_0090,reactivity_0091,reactivity_0092,reactivity_0093,reactivity_0094,reactivity_0095,reactivity_0096,reactivity_0097,reactivity_0098,reactivity_0099,reactivity_0100,reactivity_0101,reactivity_0102,reactivity_0103,reactivity_0104,reactivity_0105,reactivity_0106,reactivity_0107,reactivity_0108,reactivity_0109,reactivity_0110,reactivity_0111,reactivity_0112,reactivity_0113,reactivity_0114,reactivity_0115,reactivity_0116,reactivity_0117,reactivity_0118,reactivity_0119,reactivity_0120,reactivity_0121,reactivity_0122,reactivity_0123,reactivity_0124,reactivity_0125,reactivity_0126,reactivity_0127,reactivity_0128,reactivity_0129,reactivity_0130,reactivity_0131,reactivity_0132,reactivity_0133,reactivity_0134,reactivity_0135,reactivity_0136,reactivity_0137,reactivity_0138,reactivity_0139,reactivity_0140,reactivity_0141,reactivity_0142,reactivity_0143,reactivity_0144,reactivity_0145,reactivity_0146,reactivity_0147,reactivity_0148,reactivity_0149,reactivity_0150,reactivity_0151,reactivity_0152,reactivity_0153,reactivity_0154,reactivity_0155,reactivity_0156,reactivity_0157,reactivity_0158,reactivity_0159,reactivity_0160,reactivity_0161,reactivity_0162,reactivity_0163,reactivity_0164,reactivity_0165,reactivity_0166,reactivity_0167,reactivity_0168,reactivity_0169,reactivity_0170,reactivity_error_0001,reactivity_error_0002,reactivity_error_0003,reactivity_error_0004,reactivity_error_0005,reactivity_error_0006,reactivity_error_0007,reactivity_error_0008,reactivity_error_0009,reactivity_error_0010,reactivity_error_0011,reactivity_error_0012,reactivity_error_0013,reactivity_error_0014,reactivity_error_0015,reactivity_error_0016,reactivity_error_0017,reactivity_error_0018,reactivity_error_0019,reactivity_error_0020,reactivity_error_0021,reactivity_error_0022,reactivity_error_0023,reactivity_error_0024,reactivity_error_0025,reactivity_error_0026,reactivity_error_0027,reactivity_error_0028,reactivity_error_0029,reactivity_error_0030,reactivity_error_0031,reactivity_error_0032,reactivity_error_0033,reactivity_error_0034,reactivity_error_0035,reactivity_error_0036,reactivity_error_0037,reactivity_error_0038,reactivity_error_0039,reactivity_error_0040,reactivity_error_0041,reactivity_error_0042,reactivity_error_0043,reactivity_error_0044,reactivity_error_0045,reactivity_error_0046,reactivity_error_0047,reactivity_error_0048,reactivity_error_0049,reactivity_error_0050,reactivity_error_0051,reactivity_error_0052,reactivity_error_0053,reactivity_error_0054,reactivit

The bulk of this dataset lies in the `sequence`, `reactivity_*` and `reactivity_error_*` features. I'll take a look at the other categorical features just to get a sense of the origin of data etc. However, those features are not present in `test_sequences.csv`, so I do not want to spend too much time here.

There are 2 experiment types- DMS and 2A3. Note that in the given output, we are required to predict both DMS and 2A3 reactivity values for the sequence. Thus, we are likely to have separate models for DMS/2A3.

In [10]:
train_df['experiment_type'].value_counts()

experiment_type
2A3_MaP    821840
DMS_MaP    821840
Name: count, dtype: int64

Our training set is evenly split among `2A3` and `DMS`.

In [11]:
train_df['dataset_name'].value_counts()

dataset_name
DasLabBigLib_OneMil_OpenKnot_Round_2_train_DMS                                       125123
DasLabBigLib_OneMil_OpenKnot_Round_2_train_2A3                                       125123
OpenKnot1_Twist_2A3_EternaPlayers                                                     82172
OpenKnot1_Twist_DMS_EternaPlayers                                                     82172
DasLabBigLib_OneMil_RFAM_windows_100mers_DMS                                          75656
DasLabBigLib_OneMil_RFAM_windows_100mers_2A3                                          75656
DasLabBigLib_OneMil_Coronavirus_genomes_SARS_related_betacoronaviruses_splitA_2A3     70000
DasLabBigLib_OneMil_Coronavirus_genomes_SARS_related_betacoronaviruses_splitA_DMS     70000
DasLabBigLib_OneMil_Coronavirus_genomes_SARS_related_betacoronaviruses_splitB_2A3     70000
DasLabBigLib_OneMil_Coronavirus_genomes_SARS_related_betacoronaviruses_splitB_DMS     70000
DasLabBigLib_OneMil_Coronavirus_genomes_SARS_related_betacoronaviru

Most of our data are from DasLab, with PK50 coming second. I wonder if different labs will measure reactivity differently (systematic error), hence perhaps we can consider using only rows selected from certain labs. However, this may mean that our model cannot generalise as well. Something to ponder about.

In [16]:
train_df['SN_filter'].value_counts()

SN_filter
0    1205763
1     437917
Name: count, dtype: int64

From the competition information- "For evaluation, only sequences whose DMS_MaP and 2A3_MaP profiles both pass this filter will be used to score submissions."

Although `SN_filter == 0` makes up 3/4 of the training data, I suspect that it will be wise to keep only those rows where `SN_filter == 1`. This can ensure a more rigorously selected training dataset, while speeding up the training process (given my likely lack of compute).

### Keep only rows where `SN_filter == 1`

In [17]:
filtered_train_df = train_df[train_df['SN_filter'] == 1]

In [18]:
f_2A3_train_df = filtered_train_df[filtered_train_df['experiment_type'] == '2A3_MaP']
f_DMS_train_df = filtered_train_df[filtered_train_df['experiment_type'] == 'DMS_MaP']

In [22]:
print(f_2A3_train_df.shape);  print(f_DMS_train_df.shape)

(210992, 419)
(226925, 419)


We still have largely similar number of rows for each experiment type.